In [52]:
#import cv2 library for using open cv functions
import cv2

#import numpy library for using multi dimensional arrays
import numpy as np

#read image 
read_img = cv2.imread("doc18.jpg")

#shape[0] = Height of the image, shape[1] = Width of the image
#Check for Height and Width and Resize accordingly to 300PPI( DIN A4 size in pixels)
if read_img.shape[0] > read_img.shape[1]:

    img_width = 2480

    img_height = 3508

elif read_img.shape[0] < read_img.shape[1]:

    img_width = 3508

    img_height = 2480

#Preprocessing to perform edge detection 
def pre_process(image):

    #Convert into grey scale image,
    img_gy = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    #Blur the greyscale image to gaussian blur(reduces noise)
    img_br = cv2.GaussianBlur(img_gy, (5, 5), 3)

    #Perform Canny edge detection on the blur image
    #img_cy = cv2.Canny(img_br, 100,200)
    img_cy = cv2.Canny(img_br,100,200)
    
    # Morphological Transformations
    #1. Dilution 2. Erosion
    
    #Returns an array of 5 * 5, default data type = float
    #ones(shape, dtype=None, order='C')
    kernel = np.ones((4, 4))
    
    #Dilation : causes bright regions within an image to "grow" reducing the black spots,
    #Higher kernel can make it more brighter but can overwrite text.
    #cv.dilate(	src, kernel[, dst[, anchor[, iterations[, borderType[, borderValue]]]]]	) ->	dst
    img_di = cv2.dilate(img_cy, kernel, iterations=2)

    #Erosion : Erodes the boundary of the foreground object increasing the darker region
    img_th = cv2.erode(img_di, kernel, iterations=1)

    #return the image
    return img_th
    




def counters(image):

    biggest_cnt = np.array([])

    max_area = 0

    contours, hierarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:

        area = cv2.contourArea(cnt)

        if area > 2000:

            pm = cv2.arcLength(cnt, True)

            corner = cv2.approxPolyDP(cnt, 0.02*pm, True)

            if area > max_area and len(corner) == 4:

                biggest_cnt = corner

                max_area = area

    t = cv2.drawContours(img_cnt, biggest_cnt, -1, (0, 0, 255), 150)

    return biggest_cnt





def reorder(c_pts):

    c_pts = c_pts.reshape((4, 2))

    new_c_pts = np.zeros((4, 1, 2), np.int32)

    add = c_pts.sum(1)

    new_c_pts[0] = c_pts[np.argmin(add)]

    new_c_pts[3] = c_pts[np.argmax(add)]

    diff = np.diff(c_pts, axis=1)

    new_c_pts[1] = c_pts[np.argmin(diff)]

    new_c_pts[2] = c_pts[np.argmax(diff)]

    print(new_c_pts)

    return new_c_pts





def warp(image, biggest_cnt):

    biggest_cnt = reorder(biggest_cnt)

    pts1 = np.float32(biggest_cnt)

    pts2 = np.float32([[0, 0], [img_width, 0], [0, img_height], [img_width, img_height]])

    print(pts2)

    matrix = cv2.getPerspectiveTransform(pts1, pts2)

    img_trs = cv2.warpPerspective(image, matrix, (img_width, img_height))

    img_crp = img_trs[40:img_trs.shape[0]-20, 10:img_trs.shape[1]-10]

    return img_crp





img_cnt = read_img.copy()

img_rt1 = pre_process(read_img)

big_cnt = counters(img_rt1)

img_crop = warp(read_img, big_cnt)

img_f_gray = cv2.cvtColor(img_crop, cv2.COLOR_BGR2GRAY)

thresh, scan_img = cv2.threshold(img_f_gray, 120, 255, cv2.THRESH_BINARY)

cv2.imwrite("result1.png", scan_img)

cv2.imwrite("result2.png", img_rt1)

cv2.imwrite("result3.png", img_cnt)

cv2.imwrite("result4.png", img_crop)


cv2.waitKey(0)



[[[1175  344]]

 [[4394    8]]

 [[1194 2688]]

 [[4448 2932]]]
[[   0.    0.]
 [3508.    0.]
 [   0. 2480.]
 [3508. 2480.]]


-1